In [32]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

RANDOM_STATE = 42
N_JOBS = -1

models_params = {
    'xgb': {
    "n_estimators": 200,
    "max_depth": 7,
    "learning_rate": 0.05,
    "scale_pos_weight": 1,  # Аналог class_weight для XGBoost
    'n_jobs': N_JOBS,
    'random_state': RANDOM_STATE
},
    'cb': {
    "iterations": 200,  # CatBoost использует iterations вместо n_estimators
    # "depth": 7,
    # "learning_rate": 0.05,
    # "thread_count": N_JOBS, # Аналог n_jobs для CatBoost
    # "auto_class_weights": "Balanced",
    # "random_seed": RANDOM_STATE,
    }
}

xgb = XGBClassifier(model_params=models_params['xgb'])
cb = CatBoostClassifier(**models_params['cb'])

In [33]:
models = [
        ('xgb', xgb),
        ('cb', cb),
    ]

model_params ={
    'voting':'soft',
    'n_jobs':-1
}

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Имитация исходных данных
df = pd.DataFrame({
    "floor": [1, 2, 3, 4, 5, 6, 7, 8],
    "area": [40.5, 50.0, 60.2, 55.1, 70.0, 45.0, 66.6, 52.3],
    "rooms": [1, 2, 2, 3, 3, 1, 2, 2],
    "is_living": [1, 1, 1, 0, 0, 1, 0, 0]
})

# Разделение признаков и целевой переменной
X = df.drop(columns=["is_living"])
y = df["is_living"]

# Разделение на обучающую и тестовую выборки
x_train_df, x_test_df, y_train_series, y_test_series = train_test_split(X, y, test_size=0.25, random_state=42)

# Преобразование в numpy
x_train = x_train_df.to_numpy()
x_test = x_test_df.to_numpy()
y_train = y_train_series.to_numpy()
y_test = y_test_series.to_numpy()


In [35]:
from blocksnet.machine_learning.strategy.sklearn.ensemble.voting.classification_strategy import SKLearnVotingClassificationStrategy
model = SKLearnVotingClassificationStrategy(models, model_params)

In [36]:
# Приведение типов перед обучением
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train).ravel().astype(int)
y_test = np.asarray(y_test).ravel().astype(int)

model.train(x_train, y_train, x_test, y_test)


0.5

In [42]:
model.predict_proba(x_test)

array([[0.49348735, 0.50651263],
       [0.63125722, 0.36874276]])

In [43]:
model.predict(x_test)

array([1, 0])